# DEPRECATED! GO HERE FOR LATEST WORKING: <a href="../../job/space_track/process_gabbard_data_interpolated.ipynb">job/space_track/process_gabbard_data_interpolated.ipynb</a>

#### Please leave this alone though!

In [1]:
raise Exception("STOP! THIS IS DEPRECATED!")

Exception: STOP! THIS IS DEPRECATED!

In [ ]:
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm
from ipywidgets import IntProgress

# local file paths
input_file_path = f'../../../siads591 data/filtered_raw/gabbard_min.pkl.gz'
out_path = f'../../../siads591 data/processed_data/gabbard/'

# freq             frequency of datapoints after reindexing
# start_date       start date for data to start processing - remember to leave enough gap for interpolation
# interpolate_lim  limits of interpolation based on the frequency
#                  (1D frequency, 30 limit = 30 days 1 way, effectively a max of 60 days between points)
# out_file_name    output file name

freq, start_date, interpolate_lim, out_file_name = '36H', None, 20, 'gabbard_36H20.pkl'
# freq, start_date, interpolate_lim, out_file_name = '1D', '1979-10-01', 45, 'gabbard_1D_scaled_1980.pkl'
# freq, start_date, interpolate_lim, out_file_name = '1D', None, 30, 'gabbard_1D_scaled_il30.pkl'
# freq, start_date, interpolate_lim, out_file_name = '12H', '1979-10-01', 90, 'gabbard_scaled_1980.pkl'
# freq, start_date, interpolate_lim, out_file_name = '5D', None, 12, 'gabbard_5D_scaled.pkl'

In [ ]:
raw_df = pd.read_pickle(f'{input_file_path}', compression="gzip")

In [ ]:
if start_date != None:
    raw_df = raw_df[raw_df.index > start_date]

In [ ]:
display(raw_df.head(4))
display(raw_df.info())

In [ ]:
def f(source_df):
    # drop duplicated index first
    source_df = source_df[~source_df.index.duplicated(keep='first')]
    
    # Create new dataframe with index based on frequency
    new_date_index = pd.date_range(start=source_df.index.ceil(freq=freq).min(), end=source_df.index.max(), freq=freq)
    new_df = pd.DataFrame({}, index=new_date_index)
    new_df['NORAD_CAT_ID'] = source_df.NORAD_CAT_ID[0]
    
    # Combine with raw and perform interpolation
    reindexed_df = pd.concat([source_df,new_df]).sort_index()
    reindexed_df["PERIOD_x500"] = reindexed_df["PERIOD_x500"].astype(np.float32).interpolate(method='time', limit_area="inside", limit=interpolate_lim, limit_direction='both').round(0).astype(pd.UInt16Dtype())
    reindexed_df["APOAPSIS_x20"] = reindexed_df["APOAPSIS_x20"].astype(np.float32).interpolate(method='time', limit_area="inside", limit=interpolate_lim, limit_direction='both').round(0).astype(pd.UInt16Dtype())
    reindexed_df["PERIAPSIS_x20"] = reindexed_df["PERIAPSIS_x20"].astype(np.float32).interpolate(method='time', limit_area="inside", limit=interpolate_lim, limit_direction='both').round(0).astype(pd.UInt16Dtype())
    
    # get rid of the source data
    reindexed_df = reindexed_df.drop(source_df.index)

    return reindexed_df

In [ ]:
tqdm.pandas(desc="Interpolate progress")

# create a new dataframe with min/max epoch of each satellite
df = raw_df.groupby(by="NORAD_CAT_ID").progress_apply(f)
df.index = df.index.droplevel()

In [ ]:
df.dropna(inplace=True)
display(df.head(4))
display(df.info())

In [ ]:
df.to_pickle(f"{out_path}{out_file_name}")

In [ ]:
df.index.value_counts().plot()